In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
import numpy as np

In [3]:
# emaxmine data
print('Loading data...')
df = pd.read_csv('Data/data_incl_demographics filtered_SMALL.csv')


Loading data...


In [4]:
df.head()

,Unnamed: 0,V1,worry,chosen_emotion,text_long,self_rating_general,eng_native,ntok_long,nchar_long,cld_lang_long,...,prop_punct,age,Country of Birth,Current Country of Residence,Employment Status,First Language,Nationality,Sex,Social-Media,Student Status
0,1,1,low,Sadness,It is less an much an issue of how it affects ...,7,Yes,110,592,en,...,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,high,Anxiety,I am concerned that the true impact of the cur...,9,Yes,257,1424,en,...,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,med,Relaxation,"Personally, I am fairly calm about the corona ...",7,No but I speak it fluently,132,631,en,...,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,med,Relaxation,In this very moment as I am fortunate to be ab...,8,Yes,132,713,en,...,0.02,28.0,Belgium,United Kingdom,Full-Time,English,France,Female,"Facebook, Youtube, Twitter, Linkedin",No
4,5,5,med,Anxiety,I am more worried about getting access to my n...,8,Yes,132,662,en,...,0.02,57.0,United Kingdom,United Kingdom,"Not in paid work (e.g. homemaker', 'retired or...",English,United Kingdom,Female,Youtube,No


In [52]:
len(df) - 58

2425

In [32]:
np.count_nonzero(df["Sex"]=="Male") + np.count_nonzero(df["Sex"]=="Female")

2428

In [59]:
# 51 are NA
df["Sex"].value_counts()

Female               1585
Male                  843
DATA EXPIRED            2
CONSENT REVOKED         1
Prefer not to say       1
Name: Sex, dtype: int64

In [111]:
male800 = df[df["Sex"] == "Male"].sample(800)
female800 = df[df["Sex"] == "Female"].sample(800)

In [80]:
male800.head()

,Unnamed: 0,V1,worry,chosen_emotion,text_long,self_rating_general,eng_native,ntok_long,nchar_long,cld_lang_long,...,prop_punct,age,Country of Birth,Current Country of Residence,Employment Status,First Language,Nationality,Sex,Social-Media,Student Status
1871,1878,1878,high,Anxiety,Currently I have an array of emotion. I feel a...,9,Yes,316,1547,en,...,0.01,31.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,"Facebook, Twitter, Linkedin, Youtube",No
716,722,722,high,Fear,At the moment I feel confused and worried by t...,9,Yes,128,639,en,...,0.01,40.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,Instagram,No
213,219,219,high,Anxiety,I am anxious about going out incase I catch it...,6,Yes,117,524,en,...,0.02,60.0,United Kingdom,United Kingdom,Part-Time,English,United Kingdom,Female,"Facebook, Twitter",No
1849,1856,1856,high,Anxiety,I am bewildered to be honest. Anxious about my...,7,Yes,113,576,en,...,0.02,42.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,"Twitter, Tumblr, Reddit, Instagram",No
173,178,178,med,Sadness,I feel sad. I feel sorry for those who are tre...,8,Yes,122,578,en,...,0.02,51.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,Youtube,No


In [113]:
# trainingMale = male800.sample(frac=.75, random_state=0)
# testMale = male800.drop(trainingMale.index)
# trainingFemale = female800.sample(frac=.75, random_state=0)
# testFemale = female800.drop(female800.index)

training = pd.concat([trainingFemale, trainingMale])

In [114]:
training.head()

,Unnamed: 0,V1,worry,chosen_emotion,text_long,self_rating_general,eng_native,ntok_long,nchar_long,cld_lang_long,...,prop_punct,age,Country of Birth,Current Country of Residence,Employment Status,First Language,Nationality,Sex,Social-Media,Student Status
1694,1701,1701,med,Anger,I am annoyed at the blase and irresponsible at...,9,Yes,138,687,en,...,0.01,24.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,"Facebook, Youtube, Twitter, Instagram, Reddit",No
1278,1285,1285,high,Anxiety,I currently feel very worried and scared about...,7,Yes,105,548,en,...,0.01,24.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,Instagram,No
1684,1691,1691,med,Relaxation,I am not feeling anything major with respect t...,5,Yes,114,564,en,...,0.02,26.0,United Kingdom,United Kingdom,Part-Time,Turkish,United Kingdom,Female,"Facebook, Twitter, Youtube, Instagram, Reddit",Yes
835,841,841,high,Anxiety,I am fed up. Mainly because I'm working from h...,7,Yes,119,599,en,...,0.02,30.0,United Kingdom,United Kingdom,Full-Time,English,Ireland,Female,"Facebook, Youtube, Twitter, Reddit",No
146,151,151,med,Sadness,I am feeling sad about the social distancing. ...,8,Yes,142,668,en,...,0.02,34.0,United Kingdom,United Kingdom,Other,English,United Kingdom,Female,"Facebook, Youtube, Twitter",No


In [116]:
# shuffle the training dataset
training = training.sample(frac=1).reset_index(drop=True)

# create data and labels
trainingData = training["text_long"]
trainingLables = training["Sex"]

In [118]:
# lets try to predict male or female based on the text_long
training.head()


,Unnamed: 0,V1,worry,chosen_emotion,text_long,self_rating_general,eng_native,ntok_long,nchar_long,cld_lang_long,...,prop_punct,age,Country of Birth,Current Country of Residence,Employment Status,First Language,Nationality,Sex,Social-Media,Student Status
0,1444,1444,high,Anxiety,I am very anxious about contracting Covid-19 a...,9,Yes,120,618,en,...,0.02,27.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,Reddit,No
1,2405,2405,high,Sadness,I am worried it will delay my summer plans. As...,7,Yes,141,692,en,...,0.01,25.0,United Kingdom,United Kingdom,Part-Time,English,United Kingdom,Male,"Facebook, Youtube, Twitter, Instagram, Vine.co",No
2,784,784,high,Fear,I feel like myself and my close friends and fa...,9,Yes,145,692,en,...,0.01,31.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,Pinterest,No
3,1951,1951,high,Anxiety,I am very anxious about the Corona situation a...,4,Yes,205,1062,en,...,0.01,48.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,"Facebook, Youtube, Twitter, Linkedin, Pinteres...",No
4,1223,1223,high,Anxiety,I feel that countries are trying to work toget...,8,Yes,113,535,en,...,0.01,46.0,United Kingdom,United Kingdom,Full-Time,English,United Kingdom,Female,Instagram,No


In [106]:
maxlen = 0
for sentence in training["text_long"]:
    if (len(sentence) > maxlen):
        maxlen = len(sentence)
        
maxlen

2768

In [130]:
# set parameters for keras
max_features = 20000
#maxlen = 100
batch_size = 32
embedding_dims = 100
filters = 250
kernel_size = 3
hidden_dims = 10
epochs = 10

In [123]:
traingData = training["text_long"]
labelData = training["Sex"] == "Male"


In [124]:
# train test split
x_train, x_test, y_train, y_test = train_test_split( traingData, 
                                                    labelData, 
                                                    test_size=0.2, 
                                                    random_state=19)

In [125]:
# tokenize based on train dataset
tokenizer = Tokenizer(max_features)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
vocab_size = len(tokenizer.word_index) + 1 

In [126]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [131]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Build model...
Epoch 1/10
30/30 [==============================] - 40s 1s/step - loss: 0.6931 - accuracy: 0.4990 - val_loss: 0.6932 - val_accuracy: 0.4750
Epoch 2/10
30/30 [==============================] - 40s 1s/step - loss: 0.6862 - accuracy: 0.5115 - val_loss: 0.6952 - val_accuracy: 0.4750
Epoch 3/10
30/30 [==============================] - 40s 1s/step - loss: 0.6697 - accuracy: 0.5250 - val_loss: 0.6828 - val_accuracy: 0.4750
Epoch 4/10
30/30 [==============================] - 40s 1s/step - loss: 0.6473 - accuracy: 0.5875 - val_loss: 0.6694 - val_accuracy: 0.6000
Epoch 5/10
30/30 [==============================] - 40s 1s/step - loss: 0.6160 - accuracy: 0.7240 - val_loss: 0.6547 - val_accuracy: 0.5458
Epoch 6/10
30/30 [==============================] - 40s 1s/step - loss: 0.5777 - accuracy: 0.7635 - val_loss: 0.6434 - val_accuracy: 0.5875
Epoch 7/10
30/30 [==============================] - 40s 1s/step - loss: 0.5248 - accuracy: 0.8281 - val_loss: 0.6547 - val_accuracy: 0.5583
Epoch